<a href="https://colab.research.google.com/github/rufous86/mipt_hackathon_2023_BBB/blob/main/BBB_hackaton_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install implicit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 56.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import scipy.sparse as sp
from scipy.sparse import csr_matrix

import implicit

In [23]:
PATH = '/content/drive/MyDrive/Study/MIPT_magistery/semester2/hackathon'

In [3]:
df = pd.read_parquet(f'{PATH}/data/train_mfti.parquet')

# Part 1

Превратим исходные данные в таблицу различных паттернов действий пользователей

In [4]:
df_coo = df.pivot_table(values='event_timestamp', index=['cookie_id', 'vacancy_id_'], columns='event_type', aggfunc='count')
df_coo = df_coo.fillna(0)
df_coo[df_coo > 1] = 1
df_coo

event_type                                    click_contacts  click_favorite  \
cookie_id                        vacancy_id_                                   
0000c4548c3944c08972bbdc1fa4eb85 137659                  0.0             0.0   
                                 153975                  0.0             0.0   
                                 174953                  0.0             1.0   
                                 176171                  0.0             0.0   
                                 182445                  0.0             0.0   
...                                                      ...             ...   
ffffdb17f45b4032b386d691d52e6c00 107125                  1.0             0.0   
                                 107893                  0.0             0.0   
                                 150843                  0.0             0.0   
                                 160164                  0.0             0.0   
                                 185412                  1.0             0.0   

event_type                                    click_phone  click_response  \
cookie_id                        vacancy_id_                                
0000c4548c3944c08972bbdc1fa4eb85 137659               0.0             0.0   
                                 153975               0.0             0.0   
                                 174953               0.0             0.0   
                                 176171               0.0             0.0   
                                 182445               0.0             0.0   
...                                                   ...             ...   
ffffdb17f45b4032b386d691d52e6c00 107125               0.0             0.0   
                                 107893               0.0             0.0   
                                 150843               0.0             0.0   
                                 160164               0.0             0.0   
                                 185412               0.0             0.0   

event_type                                    preview_click_contacts  \
cookie_id                        vacancy_id_                           
0000c4548c3944c08972bbdc1fa4eb85 137659                          0.0   
                                 153975                          0.0   
                                 174953                          0.0   
                                 176171                          0.0   
                                 182445                          0.0   
...                                                              ...   
ffffdb17f45b4032b386d691d52e6c00 107125                          0.0   
                                 107893                          1.0   
                                 150843                          1.0   
                                 160164                          1.0   
                                 185412                          0.0   

event_type                                    preview_click_favorite  \
cookie_id                        vacancy_id_                           
0000c4548c3944c08972bbdc1fa4eb85 137659                          0.0   
                                 153975                          0.0   
                                 174953                          0.0   
                                 176171                          0.0   
                                 182445                          0.0   
...                                                              ...   
ffffdb17f45b4032b386d691d52e6c00 107125                          0.0   
                                 107893                          0.0   
                                 150843                          0.0   
                                 160164                          0.0   
                                 185412                          0.0   

event_type                                    preview_click_phone  \
cookie_id     

Проанализировав каждый паттерн дадим оценку возможного следующего действия пользователя в каждом отдельном взаимодействии

In [5]:
df_coo = df_coo.reset_index()
df_index = df_coo[['cookie_id', 'vacancy_id_']]
df_values = df_coo[['click_response', 'preview_click_response', 
                'click_contacts', 'click_favorite',
                'click_phone', 'preview_click_contacts',
                'preview_click_favorite', 'preview_click_phone',
                'preview_click_vacancy', 'show_vacancy']]

display(df_index.head(3))
display(df_values.head(3))

event_type,cookie_id,vacancy_id_
0,0000c4548c3944c08972bbdc1fa4eb85,137659
1,0000c4548c3944c08972bbdc1fa4eb85,153975
2,0000c4548c3944c08972bbdc1fa4eb85,174953


event_type,click_response,preview_click_response,click_contacts,click_favorite,click_phone,preview_click_contacts,preview_click_favorite,preview_click_phone,preview_click_vacancy,show_vacancy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0


In [6]:
user_item_data = csr_matrix(df_values.values)

Bayesian Personalized Ranking (BPR) - это модель рекомендаций, которая используется для ранжирования элементов в персонализированном списке рекомендаций. BPR использует методы байесовского вывода для обучения модели, которая может ранжировать элементы в списке на основе предпочтений пользователя. Он был представлен в 2009 году статьей "BPR: Bayesian Personalized Ranking from Implicit Feedback".

Для обучения модели BPR использует неявную обратную связь, чтобы предсказать, какой элемент пользователь предпочтет. Обратная связь считается неявной, потому что она не содержит явной оценки для элемента. 

BPR использует матричное разложение для извлечения скрытых признаков элементов и пользователей. Он моделирует относительные ранги элементов в списке рекомендаций, а не абсолютные оценки.

Преимущества использования BPR включают:

- BPR может работать с неявной обратной связью, что делает его более универсальным и полезным для больших наборов данных.
- BPR может работать с большим количеством элементов в списке рекомендаций, что делает его особенно полезным для списков рекомендаций, которые могут иметь тысячи или миллионы элементов.

Однако, BPR также имеет свои недостатки:

- BPR не использует явных оценок элементов, что может привести к менее точным рекомендациям, особенно для новых пользователей, у которых еще нет истории покупок.
- BPR может быть более сложным для понимания и реализации, чем другие модели рекомендаций.

In [7]:
model = implicit.bpr.BayesianPersonalizedRanking()
model.fit(user_item_data)

  0%|          | 0/100 [00:00<?, ?it/s]

После обучения модели собираем обратно матрицу действий пользователя, добавив в нее потенциальные действия, полученные на прошлом шаге (гиперпараметр N=4 указывает на то, что нас интересуют только топ 4 предполагаемых интересов пользователя, при этом мы устанавливаем флаг filter_already_liked_items=False, чтобы не потерять информацию об уже совершенных действиях)

In [8]:
data = []
for i in tqdm(df_values.index):
    userid = i
    recommendations = model.recommend(userid,
                                      user_item_data[userid],
                                      N=4,
                                      filter_already_liked_items=False)
    pred_values = [0] * 10
    for idx, i in enumerate(recommendations[0]):
        pred_values[i] = 9 - idx
    data.append(pred_values)

  0%|          | 0/4678585 [00:00<?, ?it/s]

In [9]:
df_after_BPR = pd.DataFrame(data=data, columns=df_values.columns)
df_after_BPR[['cookie_id', 'vacancy_id_']] = df_index[['cookie_id', 'vacancy_id_']]
new_order = [ 'cookie_id', 'vacancy_id_', 'click_response',
        'preview_click_response', 'click_contacts', 'preview_click_contacts',
       'click_favorite', 'preview_click_favorite', 'click_phone', 
        'preview_click_phone', 'show_vacancy', 'preview_click_vacancy']
df_after_BPR = df_after_BPR[new_order]
df_after_BPR = df_after_BPR.set_index(['cookie_id', 'vacancy_id_'])
df_after_BPR

event_type                                    click_response  \
cookie_id                        vacancy_id_                   
0000c4548c3944c08972bbdc1fa4eb85 137659                    0   
                                 153975                    0   
                                 174953                    7   
                                 176171                    0   
                                 182445                    0   
...                                                      ...   
ffffdb17f45b4032b386d691d52e6c00 107125                    0   
                                 107893                    0   
                                 150843                    0   
                                 160164                    0   
                                 185412                    0   

event_type                                    preview_click_response  \
cookie_id                        vacancy_id_                           
0000c4548c3944c08972bbdc1fa4eb85 137659                            7   
                                 153975                            7   
                                 174953                            0   
                                 176171                            7   
                                 182445                            7   
...                                                              ...   
ffffdb17f45b4032b386d691d52e6c00 107125                            7   
                                 107893                            9   
                                 150843                            9   
                                 160164                            9   
                                 185412                            7   

event_type                                    click_contacts  \
cookie_id                        vacancy_id_                   
0000c4548c3944c08972bbdc1fa4eb85 137659                    0   
                                 153975                    0   
                                 174953                    0   
                                 176171                    0   
                                 182445                    0   
...                                                      ...   
ffffdb17f45b4032b386d691d52e6c00 107125                    0   
                                 107893                    0   
                                 150843                    0   
                                 160164                    0   
                                 185412                    0   

event_type                                    preview_click_contacts  \
cookie_id                        vacancy_id_                           
0000c4548c3944c08972bbdc1fa4eb85 137659                            0   
                                 153975                            0   
                                 174953                            0   
                                 176171                            0   
                                 182445                            0   
...                                                              ...   
ffffdb17f45b4032b386d691d52e6c00 107125                            0   
                                 107893                            8   
                                 150843                            8   
                                 160164                            8   
                                 185412                            0   

event_type                                    click_favorite  \
cookie_id                        vacancy_id_                   
0000c4548c3944c08972bbdc1fa4eb85 137659                    0   
                                 153975                    0   
                                 174953                    6   
                                 176171                    0   
                                 182445                    0   
...         

# Part 2

Нас интересует информация о 6 значимых действиях (указаны в most_important_cols). Так как по значимости они все равноценны, то присвоим каждому оценку 1.
Для определения отношения пользователя к конкретной вакансии вцелом, мы просто сложим оценки всех значимых для нас действий
В итоге у нас получится матрица отношения пользователя к каждой вакансии, с которой ему пришлось взаимодействовать. При чем с учётом предположения о возможном будущем взаимодействии с этой вакансией

In [10]:
# marking

df_train = df_after_BPR.copy()

df_train[df_train > 1] = 1

most_important_cols = ['click_response', 'preview_click_response',
                       'click_contacts', 'preview_click_contacts',
                       'click_phone', 'preview_click_phone']


df_train['mark'] = df_train[most_important_cols].sum(axis=1)
df_train = df_train['mark'].reset_index()

dict_cookie_id = {}
i = 0
for cookie_id in df_train['cookie_id'].unique():
    dict_cookie_id[cookie_id] = i
    i += 1

df_train['cookie_id'] = df_train['cookie_id'].apply(lambda x: dict_cookie_id[x])
df_train

,cookie_id,vacancy_id_,mark
0,0,137659,1
1,0,153975,1
2,0,174953,1
3,0,176171,1
4,0,182445,1
...,...,...,...
4678580,330179,107125,1
4678581,330179,107893,3
4678582,330179,150843,3
4678583,330179,160164,3


Далее мы создаём вновь спарс матрицу и обучаем ее на алгоритме ALS, после чего проверяем на точность предсказания топ-5 рекомендованных вакансий (используемая метрика precision@5)

In [11]:
data = df_train['mark']
row = df_train['cookie_id']
col = df_train['vacancy_id_']
# convert DataFrame to COO sparse matrix
user_item_data = sp.coo_matrix((data, (row, col))).tocsr()

In [12]:
df_test = pd.read_parquet(f'{PATH}/data/test_public_mfti.parquet')
df_test.head()

,cookie_id,vacancy_id_
0,000cd76cd33f43d4a1ac1d16d10f8bf7,"[222177, 222173, 222163, 238874, 238878, 22812..."
1,0034bc7f404341ba8412665453e7825a,"[102794, 137587, 257319, 237756, 240744, 11348..."
2,00a6c5a64a274c55a836402bdeb3b2c4,"[254292, 164602, 116438, 228634, 218819, 24065..."
3,015937a125b14e74bdff1cddc49f9172,"[246685, 138123, 115420, 210628, 212325, 235196]"
4,01de50c280794cec8804f16f45f847b7,"[219070, 251469, 166899, 212703, 214561]"


In [13]:
def precision_at_5(model, df_test):
    precision = []
    for idx, row in df_test.iterrows():
        cookie_id = row['cookie_id']
        userid = dict_cookie_id[cookie_id]
        recommendations = model.recommend(userid, user_item_data[userid], N=5)[0]
        TP = len(set(recommendations).intersection(set(row['vacancy_id_'][:])))
        precision.append(TP / 5)

    return np.mean(precision)

result_grid = {
    'factors': [],
    'regularization': [],
    'alpha': [],
    'precision@5': [],
}

def train(factors, regularization, alpha):
    model = implicit.als.AlternatingLeastSquares(
                                        factors=factors,
                                        regularization=regularization,
                                        alpha=alpha,
                                        calculate_training_loss=True, 
                                        iterations=30)
    model.fit(user_item_data)
    precision = precision_at_5(model, df_test)
    
    result_grid['factors'].append(factors)
    result_grid['regularization'].append(regularization)
    result_grid['alpha'].append(alpha)
    result_grid['precision@5'].append(precision)
    print(f'precision@5: {precision}')

In [14]:
i = 1
for factors in [500, 1000]:
    for regularization in [0.1, 1.0]:
        for alpha in [0.1, 1.0, 2.0]:
            print(i, 'factors', factors,
                  'regularization', regularization,
                  'alpha', alpha)
            train(factors, regularization, alpha)
            print('-'*10)
            i += 1

1 factors 500 regularization 0.1 alpha 0.1


  0%|          | 0/30 [00:00<?, ?it/s]

precision@5: 0.032383419689119175
----------
2 factors 500 regularization 0.1 alpha 1.0


  0%|          | 0/30 [00:00<?, ?it/s]

precision@5: 0.03290155440414508
----------
3 factors 500 regularization 0.1 alpha 2.0


...

precision@5: 0.029533678756476684
----------
11 factors 1000 regularization 1.0 alpha 1.0


  0%|          | 0/30 [00:00<?, ?it/s]

precision@5: 0.029792746113989636
----------
12 factors 1000 regularization 1.0 alpha 2.0


  0%|          | 0/30 [00:00<?, ?it/s]

precision@5: 0.030051813471502594
----------


In [15]:
result = pd.DataFrame(result_grid).sort_values(by=['precision@5'], ascending=False)
result

,factors,regularization,alpha,precision@5
1,500,0.1,1.0,0.032902
3,500,1.0,0.1,0.032902
0,500,0.1,0.1,0.032383
4,500,1.0,1.0,0.032383
5,500,1.0,2.0,0.031865
2,500,0.1,2.0,0.031606
6,1000,0.1,0.1,0.030052
8,1000,0.1,2.0,0.030052
11,1000,1.0,2.0,0.030052
10,1000,1.0,1.0,0.029793


# Результат
### Образец

In [16]:
pd.read_parquet(f'{PATH}/data/test_private_sample_submission_mfti.parquet')

,cookie_id,predictions
0,0018914ba3e54011b28fa715583d3354,"[100100, 100101, 100102, 100103, 100104]"
1,0035c298d8c64f368ae730a9cca9bb20,"[100100, 100101, 100102, 100103, 100104]"
2,00956458877448ec9fba87fb97443fdf,"[100100, 100101, 100102, 100103, 100104]"
3,0099387c921b41e7bae6c99dd8254b60,"[100100, 100101, 100102, 100103, 100104]"
4,009f65e8ae99413a8da94a491320580a,"[100100, 100101, 100102, 100103, 100104]"
...,...,...
3081,ffadd195859444d2ade2479b0611c5c1,"[100100, 100101, 100102, 100103, 100104]"
3082,ffbc08b528c64f22996873fc63872202,"[100100, 100101, 100102, 100103, 100104]"
3083,ffdeaf3c34544529880aebf17c103f6c,"[100100, 100101, 100102, 100103, 100104]"
3084,ffefa79a74804ee69e6c131e0d05b948,"[100100, 100101, 100102, 100103, 100104]"


In [17]:
private_test_df = pd.read_parquet(f'{PATH}/data/test_private_users_mfti.parquet')
private_test_df

,cookie_id
0,0018914ba3e54011b28fa715583d3354
1,0035c298d8c64f368ae730a9cca9bb20
2,00956458877448ec9fba87fb97443fdf
3,0099387c921b41e7bae6c99dd8254b60
4,009f65e8ae99413a8da94a491320580a
...,...
3081,ffadd195859444d2ade2479b0611c5c1
3082,ffbc08b528c64f22996873fc63872202
3083,ffdeaf3c34544529880aebf17c103f6c
3084,ffefa79a74804ee69e6c131e0d05b948


In [18]:
best_features = result.head(1).iloc[0]
print(best_features)

model = implicit.als.AlternatingLeastSquares(
                                factors=best_features['factors'],
                                regularization=best_features['regularization'],
                                alpha=best_features['alpha'],
                                calculate_training_loss=True, 
                                iterations=30)
model.fit(user_item_data)

factors           500.000000
regularization      0.100000
alpha               1.000000
precision@5         0.032902
Name: 1, dtype: float64


  0%|          | 0/30 [00:00<?, ?it/s]

### Рекомендации

In [21]:
rec_dict = {
    'cookie_id': [],
    'predictions': []    
}

for idx, row in private_test_df.iterrows():
    cookie_id = row['cookie_id']
    userid = dict_cookie_id[cookie_id]
    recommendations = model.recommend(userid, user_item_data[userid], N=5)[0]
    rec_dict['cookie_id'].append(cookie_id)
    rec_dict['predictions'].append(list(recommendations))

recommendation_df = pd.DataFrame(rec_dict)
recommendation_df

,cookie_id,predictions
0,0018914ba3e54011b28fa715583d3354,"[182382, 155615, 154656, 136514, 139451]"
1,0035c298d8c64f368ae730a9cca9bb20,"[126019, 135430, 132616, 153586, 171691]"
2,00956458877448ec9fba87fb97443fdf,"[154356, 246285, 219770, 119161, 154498]"
3,0099387c921b41e7bae6c99dd8254b60,"[173015, 202969, 135584, 125869, 197407]"
4,009f65e8ae99413a8da94a491320580a,"[120188, 227207, 107526, 133162, 121955]"
...,...,...
3081,ffadd195859444d2ade2479b0611c5c1,"[253946, 127706, 105190, 125869, 113670]"
3082,ffbc08b528c64f22996873fc63872202,"[189317, 235614, 206907, 107830, 133079]"
3083,ffdeaf3c34544529880aebf17c103f6c,"[181976, 178208, 213822, 173148, 216952]"
3084,ffefa79a74804ee69e6c131e0d05b948,"[146630, 218223, 246815, 212953, 128484]"


In [24]:
recommendation_df.to_parquet(f'{PATH}/data/submission_mfti_BBB.parquet', index=False)
pd.read_parquet(f'{PATH}/data/submission_mfti_BBB.parquet')

,cookie_id,predictions
0,0018914ba3e54011b28fa715583d3354,"[182382, 155615, 154656, 136514, 139451]"
1,0035c298d8c64f368ae730a9cca9bb20,"[126019, 135430, 132616, 153586, 171691]"
2,00956458877448ec9fba87fb97443fdf,"[154356, 246285, 219770, 119161, 154498]"
3,0099387c921b41e7bae6c99dd8254b60,"[173015, 202969, 135584, 125869, 197407]"
4,009f65e8ae99413a8da94a491320580a,"[120188, 227207, 107526, 133162, 121955]"
...,...,...
3081,ffadd195859444d2ade2479b0611c5c1,"[253946, 127706, 105190, 125869, 113670]"
3082,ffbc08b528c64f22996873fc63872202,"[189317, 235614, 206907, 107830, 133079]"
3083,ffdeaf3c34544529880aebf17c103f6c,"[181976, 178208, 213822, 173148, 216952]"
3084,ffefa79a74804ee69e6c131e0d05b948,"[146630, 218223, 246815, 212953, 128484]"
